# Importing Libraries

In [ ]:
import os
import shutil
import numpy as np
import cv2
import face_recognition
import datetime
import time

## Detecting Faces in an Image using Caffe DNN model

In [ ]:
start_time = time.time()

def detect_faces_dnn(image_path):
    # Load the image
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    # Load the pre-trained model
    prototxt_path = 'deploy.prototxt'
    model_path = 'res10_300x300_ssd_iter_140000.caffemodel'
    net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

    # Prepare the image for the DNN
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    # Extract bounding boxes
    face_locations = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face_locations.append((startY, endX, endY, startX))

    return face_locations



## Get face encodings

In [ ]:
def get_face_encodings(image_path, face_locations):
    # Load the image
    image = face_recognition.load_image_file(image_path)

    # Find all face encodings in the image
    face_encodings = face_recognition.face_encodings(image, face_locations)

    return face_encodings

## Known Faces

In [ ]:
def create_known_face_encodings(known_faces_dir):
    known_face_encodings = []
    known_face_names = []
    for image_name in os.listdir(known_faces_dir):
        if image_name.startswith('.'):
            continue

        try:
            image_path = os.path.join(known_faces_dir, image_name)
            face_locations = detect_faces_dnn(image_path)
            encodings = get_face_encodings(image_path, face_locations)
            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(os.path.splitext(image_name)[0])
            else:
                print(f"No face found in {image_name}")
        except Exception as e:
            print(f"Error processing file {image_name}: {e}")

    return known_face_encodings, known_face_names



## Face grouping

In [ ]:

def organize_event_photos(event_photos_dir, known_face_encodings, known_face_names, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
       
    for image_name in os.listdir(event_photos_dir):
        if image_name.startswith('.'):
            continue

        image_path = os.path.join(event_photos_dir, image_name)

        try:
            face_locations = detect_faces_dnn(image_path)
            face_encodings = get_face_encodings(image_path, face_locations)
            if not face_encodings:
                print(f"No faces found in {image_name}")
                continue

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                    person_dir = os.path.join(output_dir, name)
                    if not os.path.exists(person_dir):
                        os.makedirs(person_dir)
                    shutil.copy(image_path, person_dir)

        except Exception as e:
            print(f"Error processing file {image_name}: {e}")


## Defining Directories 

In [ ]:
# Define directories
known_faces_dir = 'C:/Users/.../known_faces'
event_photos_dir = 'C:/Users/.../Testing Faces'
output_dir = 'C:/Users/.../Organized Photos using DNN'


## Driver

In [ ]:
# Create known face encodings
known_face_encodings, known_face_names = create_known_face_encodings(known_faces_dir)

# Organize event photos
organize_event_photos(event_photos_dir, known_face_encodings, known_face_names, output_dir)
end_time = time.time()
print(f"Organizing completed in {end_time - start_time} seconds.")